# Image Filtering: Spatial Methods

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt

from skimage import io
from skimage import exposure
from skimage import filters as flt
from skimage.util import img_as_float32 as img_as_float

from scipy import signal
from scipy.ndimage import convolve

In [ ]:
def print_imginfo(I):
    print(type(I))
    print(I.shape, I.dtype)
    print('Data range:', np.min(I), 'to', np.max(I))

In [ ]:
def show_imghist(I, vmin=0.0, vmax=1.0):
    fig, ax = plt.subplots(1, 2, figsize=(10,3))
    
    ax[0].imshow(I, cmap='gray', vmin=vmin, vmax=vmax)
    ax[0].set_axis_off()
    
    ax[1].hist(I.ravel(), lw=0, bins=256, range=(vmin+0.01,vmax-0.01));
    ax[1].set_xlim(vmin, vmax)
    ax[1].set_yticks([])

In [ ]:
I1 = io.imread("../../images/parrot.png", as_gray=True)
#I1 = io.imread("../../images/cars.jpg", as_gray=True)
#I1 = io.imread("../../images/coins.png", as_gray=True)
I1 = img_as_float(I1)

print_imginfo(I1)
show_imghist(I1)

## Linear Filter: Derivative Kernels

Many derivative kernels are formed as the product of a smoothing kernel and a differentiation kernel. Both can often  be expressed as the convolution of two smaller kernels. As an example, consider the Sobel operator. Because convolution flips the kernels prior to use, the order of postive vs negative coeffcients is the opposite of what you might expect.

\begin{equation}
h_x = \begin{bmatrix} +1 & 0 & -1 \\ +2 & 0 & -2 \\ +1 & 0 & -1 \end{bmatrix}
    = \begin{bmatrix} 1 \\ 2 \\ 1 \end{bmatrix}  \begin{bmatrix} 1 & 0 & -1 \end{bmatrix}
    = \begin{bmatrix} 1 \\ 1 \end{bmatrix} * \begin{bmatrix} 1 \\ 1 \end{bmatrix}
      \begin{bmatrix} 1 & -1 \end{bmatrix} * \begin{bmatrix} 1 & 1 \end{bmatrix}
\end{equation}


\begin{equation}
h_y = \begin{bmatrix} +1 & +2 & +1 \\ 0 & 0 & 0 \\ -1 & -2 & -1 \end{bmatrix}
    = \begin{bmatrix} 1 \\ 0 \\ -1 \end{bmatrix}  \begin{bmatrix} 1 & 2 & 1 \end{bmatrix}
    = \begin{bmatrix} 1 \\ 1 \end{bmatrix} * \begin{bmatrix} 1 \\ -1 \end{bmatrix}
      \begin{bmatrix} 1 & 1 \end{bmatrix} * \begin{bmatrix} 1 & 1 \end{bmatrix}
\end{equation}

The kernels can be implemented as
`h1=np.array([...]), h2=np.array([...]), h=scipy.signal.convolve(h1, h2, mode='full')`

Python supports complex numbers: `C=R+Ij`. By letting $h_x$ and $h_y$ be the real and imaginary parts of a complex derivative kernel, convolution produces a complex image that holds all the information needed to determine the gradient magnitudes and angles. See details below.

In [ ]:
# Center pixel weighted more heavily when smoothing
def sobel_kernel():
    h = np.array([ [1+1j,0+2j,-1+1j], [2+0j,0+0j,-2+0j], [1-1j,0-2j,-1-1j] ])
    return h

# All pixels weighted the same when smoothing
def prewitt_kernel():
    h = np.array([ [1+1j,0+1j,-1+1j], [1+0j,0+0j,-1+0j], [1-1j,0-1j,-1-1j] ])
    return h

# Add horizontal and vertical central differences to pixel value (gives illusion of depth)
def emboss_kernel():
    h = np.array([ [2,1,0], [1,1,-1], [0,-1,-2]])
    return h

# Second order derivative (aka negative Laplacian)
def laplacian_kernel():
    h = np.array([ [0,-1,0], [-1,4,-1], [0,-1,0] ])
    return h

# Second order derivative w/Gaussian smoothing
def mexicanhat_kernel(sigma=1.0, truncate=3.0):
    N = np.int32(np.round(truncate*sigma))
    
    x = np.arange(-N, N+1)
    y = np.arange(-N, N+1)
    xv, yv = np.meshgrid(x, y)
    
    h = np.exp(-0.5*(xv**2+yv**2)/(sigma**2))
    h *= 1/(np.pi*sigma**4)*(1 - 0.5*(xv**2+yv**2)/(sigma**2))

    return h

## Linear Filter: Image Gradients

In [ ]:
# mode: reflect  (d c b a | a b c d | d c b a)
# mode: constant (k k k k | a b c d | k k k k) -- (cval=k)
# mode: nearest  (a a a a | a b c d | d d d d)
# mode: mirror     (d c b | a b c d | c b a)
# mode: wrap     (a b c d | a b c d | a b c d)

mode = 'reflect'

Ip = I1
Ip = flt.gaussian(Ip, sigma=2, mode=mode)

h = sobel_kernel()
#h = prewitt_kernel()

I2 = convolve(Ip, h, mode=mode)

show_imghist(np.real(I2), -0.2, 0.2)      # horizontal gradient
show_imghist(np.imag(I2), -0.2, 0.2)      # vertical gradient
show_imghist(np.absolute(I2), 0, 1.0)     # gradient magnitude
show_imghist(np.angle(I2), -np.pi, np.pi)   # gradient angle

## Linear Filter: Embossing

In [ ]:
h = emboss_kernel()

I2 = convolve(I1, h, mode=mode)

print_imginfo(I2)

show_imghist(I1, vmin=0, vmax=1)
show_imghist(I2, vmin=0, vmax=1)

## Linear Filter: Unsharp Masking

In [ ]:
sigma = 1.0
truncate = 3.0
alpha = 3.0

Ip = I1
Ip = flt.gaussian(Ip, sigma=sigma, truncate=truncate, mode=mode)

h = laplacian_kernel()
I2 = convolve(Ip, h, mode=mode)

I3 = I1 + alpha*I2

show_imghist(I1)
show_imghist(I2, vmin=-0.04, vmax=0.04)
show_imghist(I3)

In [ ]:
h = mexicanhat_kernel(sigma=sigma, truncate=truncate)
I2 = convolve(I1, h, mode=mode)

I3 = I1 + alpha*I2

show_imghist(I1)
show_imghist(I2, vmin=-0.04, vmax=0.04)
show_imghist(I3)